In [1]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from docx2python import docx2python
import os
from sklearn.metrics.pairwise import cosine_similarity

GOOGLE_API_KEY="YOUR API GOES HERE"
genai.configure(api_key=GOOGLE_API_KEY)
model = "models/embedding-001"

Para utilizar a função abaixo e extrair o conteúdo de documentos .docx de um diretório específico, basta fornecer o caminho para esse diretório como argumento da função.

In [2]:
def extract_docx_contents(docs_path):
    """
    Extracts the content of .docx documents from a specific directory.

    Args:
        docs_path (str): The path to the directory containing the .docx documents.

    Returns:
        list: A list of dictionaries, where each dictionary contains the title and content of a document.

    Example:
        >>> docs_path = "/path/to/your/docs"
        >>> documents = extract_docx_contents(docs_path)
    """

    documents = []

    for doc in os.listdir(docs_path):
        if doc.endswith(".docx"):
            path_file = os.path.join(docs_path, doc)
            doc_title = os.path.splitext(doc)[0]
            docx_content = docx2python(path_file)
           
            if docx_content.body and len(docx_content.body) > 1:
                text = ''.join([''.join(subsublist) for sublist in docx_content.body[1] for subsublist in sublist])
            elif docx_content.body:
                text = docx_content.text

            document = {
                    "Title": doc_title,
                    "Content": text
                }
            documents.append(document)
            

    return documents


O seguinte bloco de código realiza a pré-processamento dos dados extraídos dos documentos .docx. Isso inclui a formatação dos títulos e conteúdos dos documentos para melhor organização e análise.


In [3]:
docs_path = "my_docs"
documents = extract_docx_contents(docs_path)
df = pd.DataFrame(documents)
df.columns = ["Title", "Content"]
df["Content"] = df["Content"].str.replace(r"\t|\n", "", regex=True)
df

,Title,Content
0,Basquetebol,"O basquetebol ou bola ao cesto, popularmente c..."
1,Brasil,"Brasil (localmente [bɾaˈziw][b]), oficialmente..."
2,Culinaria Japonesa,A culinária japonesa se desenvolveu ao longo d...
3,Culinaria,"A culinária é a arte de cozinhar,[1] (ver: art..."
4,F1,Fórmula 1 (também F1; em inglês: Formula One) ...
5,Futebol,"O futebol,[4] também referido como futebol de ..."
6,Japao,"Coordenadas: 35° 9' 22"" N 136° 3' 36"" ECapital..."


In [4]:
def embed_fn(title, text):
  """
    Generates an embedding for the provided title and text using a specific model.

    This function utilizes Google Generative AI to generate an embedding for the provided title and text.
    The resulting embedding is useful for document retrieval tasks, where documents can be compared
    based on the similarity of their embeddings.

    Args:
        title (str): The title of the document.
        text (str): The text of the document.

    Returns:
        numpy.ndarray: The embedding generated for the provided title and text.

    Example:
        >>> title = "Example Title"
        >>> text = "This is an example text."
        >>> embedding = embed_fn(title, text)
    """

  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

A linha abaixo utiliza a função `embed_fn` para gerar embeddings para o título e o conteúdo de cada documento presente no DataFrame `df`. Esses embeddings são armazenados na coluna "Embeddings" do DataFrame para posterior análise ou processamento.


In [5]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Title"], row["Content"]), axis=1)
df

,Title,Content,Embeddings
0,Basquetebol,"O basquetebol ou bola ao cesto, popularmente c...","[0.03007518, -0.024216983, -0.020793779, -0.00..."
1,Brasil,"Brasil (localmente [bɾaˈziw][b]), oficialmente...","[0.041011292, -0.014829685, -0.023700345, 0.06..."
2,Culinaria Japonesa,A culinária japonesa se desenvolveu ao longo d...,"[0.021962343, -0.034408245, -0.05993954, 0.034..."
3,Culinaria,"A culinária é a arte de cozinhar,[1] (ver: art...","[0.015466951, -0.034977786, -0.03548918, 0.027..."
4,F1,Fórmula 1 (também F1; em inglês: Formula One) ...,"[0.0017117915, -0.025918707, -0.0041319, -0.01..."
5,Futebol,"O futebol,[4] também referido como futebol de ...","[0.021318262, -0.022684429, -0.059795767, 0.01..."
6,Japao,"Coordenadas: 35° 9' 22"" N 136° 3' 36"" ECapital...","[0.035828836, -0.016120639, -0.030543817, 0.03..."



---

## Buscas

Para simplificar a busca dentro dos documentos, o argumento `query` nas funções abaixo pode ser um trecho específico presente nos arquivos, uma parte do documento ou até mesmo algo relacionado ao tópico que você está procurando.

Essas funções foram projetadas para facilitar a localização de documentos relevantes com base em consultas específicas. Basta fornecer um termo ou frase relevante e as funções encontrarão os documentos mais adequados dentro da base de dados.

---


In [6]:
def generate_and_search_query(query, database, model):
    """
    Generates and searches a query within a given database using a specified model.

    This function generates an embedding for the provided query using the specified model.
    It then compares this embedding with the embeddings of documents in the database
    to find the most relevant document.

    Args:
        query (str): The query to be searched within the database.
        database (DataFrame): The DataFrame containing the database of documents.
        model (str): The model to be used for generating embeddings.

    Returns:
        str: The content of the most relevant document to the query.

    Example:
        >>> query = "Example query"
        >>> relevant_document = generate_and_search_query(query, df, "models/embedding-001")
    """
    embedding_of_query = genai.embed_content(model=model,
                                             content=query,
                                             task_type="RETRIEVAL_QUERY")["embedding"]

    dot_products = np.dot(np.stack(database["Embeddings"]), embedding_of_query)
    index = np.argmax(dot_products)

    return database.iloc[index]["Content"]


In [7]:
def generate_and_search_top_3_docs(query, database, model, similarity_threshold=0.65):
    """
    Generates and searches a query within a given database using a specified model, returning the top 3 most relevant documents with a similarity score equal to or greater than the specified threshold.

    This function generates an embedding for the provided query using the specified model.
    It then compares this embedding with the embeddings of documents in the database
    to find the top 3 most relevant documents with a similarity score equal to or greater than the specified threshold.

    Args:
        query (str): The query to be searched within the database.
        database (DataFrame): The DataFrame containing the database of documents.
        model (str): The model to be used for generating embeddings.
        similarity_threshold (float, optional): The minimum similarity score required for a document to be considered relevant. Defaults to 0.65.

    Returns:
        list: A list of tuples containing the content of the top 3 most relevant documents to the query, along with their similarity scores.

    Example:
        >>> query = "Example query"
        >>> top_3_documents = generate_and_search_top_3_docs(query, df, "models/embedding-001")
    """
    embedding_of_query = genai.embed_content(model=model,
                                             content=query,
                                             task_type="RETRIEVAL_QUERY")["embedding"]

    embeddings_of_database = np.stack(database["Embeddings"])
    similarities = cosine_similarity(embeddings_of_database, np.array(embedding_of_query).reshape(1, -1)).flatten()
    relevant_indices = np.where(similarities >= similarity_threshold)[0]
    sorted_indices = np.argsort(similarities[relevant_indices])[::-1][:3]

    top_relevant_documents = [(database.iloc[relevant_indices[index]]["Content"], similarities[relevant_indices[index]]) for index in sorted_indices]

    return top_relevant_documents


## Exemplo de Uso das Funções de Busca

Aqui está um exemplo simples de como utilizar as funções de busca implementadas para encontrar documentos relevantes com base em consultas específicas. Abaixo, mostramos como realizar uma busca utilizando uma consulta de exemplo e as funções `generate_and_search_query` e `generate_and_search_top_3_docs`.


In [8]:
query = "coisas sobre japão"

top_3_documents = generate_and_search_top_3_docs(query, df, model)

print("Top 3 Relevant Documents:")
for i, (document, similarity) in enumerate(top_3_documents, start=1):
    print(f"Document {i}:")
    print(f"Similarity: {similarity}")
    print(document)
    print()

Top 3 Relevant Documents:
Document 1:
Similarity: 0.7226724109311793
Coordenadas: 35° 9' 22" N 136° 3' 36" ECapitalTóquio 35°41′N 139°46′ECidade mais populosaTóquioLíngua oficialJaponêsGovernoMonarquia constitucional unitária parlamentarista• ImperadorNaruhito• Primeiro-ministroFumio KishidaLegislaturaDieta Nacional • Câmara altaCâmara dos Conselheiros • Câmara baixaCâmara dos RepresentantesFundação Nacional11 de fevereiro de 660 a.C. • Constituição Meiji29 de novembro de 1890 • Constituição do Japão3 de maio de 1947 • Tratado de S. Francisco28 de abril de 1952 Área   • Total377 975[4] km² (62.º) • Água (%)0,8População   • Estimativa para 2023124 631 000[5] hab. (11.º) • Densidade337 hab./km² (30.º)PIB (base PPC)Estimativa de 2022 • TotalUS$ 6,139 trilhões *[6] (4.º) • Per capitaUS$ 49 044[6] (36.º)PIB (nominal)Estimativa de 2022 • TotalUS$ 4,234 trilhões *[6] (3.º) • Per capitaUS$ 33 822[6] (28.º)IDH (2019)0,919 (19.º) – muito alto[7]Gini (2008)37,6 (2008)[8] MoedaIene (¥ / 円) (JPY)Fu

## Integração com Gemini AI

Para integrar com o Gemini AI, configure as definições de geração da seguinte forma para que seja retornado apenas uma resposta da API. Exemplos de prompts para português e inglês foram adicionados. Para maior precisão ao documento use "Não adicione nenhuma informação extra ao seguinte trecho.". Dessa forma a genai não vai adicionar informações extras ou suposições na resposta. A API do Gemini só aceita um documento sendo enviado por vez como trecho, então selecione qual documento quer usar quando for enviar para a API.



In [9]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

prompt_br = f"Me faça um resumo do texto. Não adicione nenhuma informação extra ao seguinte trecho. Trecho: {top_3_documents[1]}"
prompt_eng = f"Ask your question here. Do not add any extra information to the following passage. Passage: {top_3_documents}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt_br)
print(response.text)

A culinária japonesa evoluiu ao longo dos séculos, influenciada por mudanças políticas e sociais. A era moderna trouxe elementos ocidentais, enquanto a culinária tradicional enfatiza a sazonalidade, a qualidade dos ingredientes e a apresentação.

As refeições tradicionais giram em torno do arroz branco, acompanhado de okazu (acompanhamentos). Os talheres são hashi (pauzinhos). As refeições são nomeadas pelo número de acompanhamentos, com ichijū-sansai (uma sopa, três acompanhamentos) sendo a mais comum.

Os frutos do mar são amplamente consumidos, assim como carne e frango. O macarrão, originário da China, é essencial, com sobá (trigo-sarraceno) e udon (trigo branco) sendo os tipos tradicionais. O ramen, uma importação mais recente, também é popular.

Em algumas regiões, insetos como gafanhotos e larvas de abelha são consumidos, assim como lagartos.
